In [1]:
# 訊息辨識模型 - 建立和訓練模型

In [2]:
# 執行資料前處理
%run 'Chinese_Texts_Data_Preprocessing.ipynb'

Using TensorFlow backend.


In [3]:
# 建立模型
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, LSTM

model = Sequential()
model.add(Embedding(output_dim=16, # 將數字list轉為32維度的向量
                    input_dim=800, # 之前建立的字典為800字
                    input_length=20)) # 每一筆資料固定長度為20
model.add(Dropout(0.20))
#model.add(Flatten()) # MLP Model
#model.add(SimpleRNN(units=16)) # RNN Model：有16個神經元
model.add(LSTM(units=16)) # LSTM Model：有32個神經元
model.add(Dropout(0.20))
model.add(Dense(units=256, # 隱藏層神經元數
                activation='relu'))
model.add(Dropout(0.20))
model.add(Dense(units = 3, # 輸出層神經元數
                kernel_initializer = 'normal',
                activation = 'softmax'))
model.compile(loss='binary_crossentropy', # loss function
              optimizer='adam', # optimizer
              metrics=['accuracy']) # other evaluation

In [4]:
# 儲存準確度最高的模型
model_path = "/Users/PChomeIM/pywork/SaveModel/Text_LSTM_Model.h5"
checkpoint = ModelCheckpoint(model_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [5]:
# 訓練模型
train_history = model.fit(train_features, train_labels, batch_size=30,
                          epochs=10, validation_split=0.2, shuffle=True, 
                          verbose=0, callbacks=callbacks_list)

Epoch 00001: val_acc improved from -inf to 0.66908, saving model to /Users/PChomeIM/pywork/SaveModel/Text_LSTM_Model.h5
Epoch 00002: val_acc improved from 0.66908 to 0.77028, saving model to /Users/PChomeIM/pywork/SaveModel/Text_LSTM_Model.h5
Epoch 00003: val_acc improved from 0.77028 to 0.81205, saving model to /Users/PChomeIM/pywork/SaveModel/Text_LSTM_Model.h5
Epoch 00004: val_acc improved from 0.81205 to 0.82892, saving model to /Users/PChomeIM/pywork/SaveModel/Text_LSTM_Model.h5
Epoch 00005: val_acc improved from 0.82892 to 0.84819, saving model to /Users/PChomeIM/pywork/SaveModel/Text_LSTM_Model.h5
Epoch 00006: val_acc did not improve
Epoch 00007: val_acc did not improve
Epoch 00008: val_acc improved from 0.84819 to 0.85060, saving model to /Users/PChomeIM/pywork/SaveModel/Text_LSTM_Model.h5
Epoch 00009: val_acc did not improve
Epoch 00010: val_acc did not improve


In [6]:
# 載入準確度最高的模型
best_model = load_model(model_path)

In [7]:
# 顯示每種標籤的預測機率
class_dic = {0:'IM 問題', 1:'EC 問題', 2:'物流問題'}
def show_predict_prob(predict_prob):
    for j in range(len(predict_prob[0])):
        print("  " + class_dic[j] + ' :%1.5f'%(predict_prob[0][j]))

In [8]:
input_text = "我買的商品什麼時候會送到？"
input_seq = token.texts_to_sequences([input_text])
pad_input_seq = sequence.pad_sequences(input_seq, maxlen=20)
predict_result = best_model.predict_classes(pad_input_seq)
# 預測結果
print("\n預測結果：" + class_dic[predict_result[0]])
print("\n每種標籤預測機率：")
# 取得每種標籤的預測機率
predict_prob = best_model.predict(pad_input_seq)
show_predict_prob(predict_prob)

1/1 [==============================] - 0s 94ms/step

預測結果：物流問題

每種標籤預測機率：
  IM 問題 :0.01870
  EC 問題 :0.06927
  物流問題 :0.91203
